In [1]:
# https://github.com/AMontgomerie/question_generator/blob/master/examples/question_generation_example.ipynb
!git clone https://github.com/amontgomerie/question_generator/
!pip3 install PyPDF2
!pip install transformers
!pip install docx2txt
!pip install sentencepiece
!pip install PyPDF2
!pip install language_tool_python

Cloning into 'question_generator'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 252 (delta 42), reused 40 (delta 39), pack-reused 188
Receiving objects: 100% (252/252), 114.54 KiB | 916.00 KiB/s, done.
Resolving deltas: 100% (129/129), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 218 kB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 
     |████████████████████████████████| 120 kB 56.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=ad52d10e008b538e5d432739faf6a4223de78e4

## Import libraries

In [2]:
import string
import time
import en_core_web_sm
import json
import numpy as np
import pandas as pd
import random
import re
import torch
import docx2txt
import requests
import PyPDF2
import nltk
nltk.download('punkt')

from nltk import sent_tokenize
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
)
from typing import Any, List, Mapping, Tuple
from transformers import AutoTokenizer, T5ForConditionalGeneration


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

## Questions Generations

In [3]:
start = time.time()

class QuestionGenerator:

    def __init__(self) -> None:

        QG_PRETRAINED = "iarfmoose/t5-base-question-generator"
        self.ANSWER_TOKEN = "<answer>"
        self.CONTEXT_TOKEN = "<context>"
        self.SEQ_LENGTH = 512

        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        self.qg_tokenizer = AutoTokenizer.from_pretrained(
            QG_PRETRAINED, use_fast=False)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(QG_PRETRAINED)
        self.qg_model.to(self.device)
        self.qg_model.eval()

        self.qa_evaluator = QAEvaluator()

    def generate(
        self,
        article: str,
        use_evaluator: bool = True,
        num_questions: bool = None,
        answer_style: str = "all"
    ) -> List:

        qg_inputs, qg_answers = self.generate_qg_inputs(article, answer_style)
        generated_questions = self.generate_questions_from_inputs(qg_inputs)

        message = "{} questions doesn't match {} answers".format(
            len(generated_questions), len(qg_answers)
        )
        assert len(generated_questions) == len(qg_answers), message

        if use_evaluator:
            print("Evaluating QA pairs...\n")
            encoded_qa_pairs = self.qa_evaluator.encode_qa_pairs(
                generated_questions, qg_answers
            )
            global scores
            scores = self.qa_evaluator.get_scores(encoded_qa_pairs)

            if num_questions:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores, num_questions
                )
            else:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores
                )

        else:
            print("Skipping evaluation step.\n")
            qa_list = self._get_all_qa_pairs(generated_questions, qg_answers)

        return qa_list

    def generate_qg_inputs(self, text: str, answer_style: str) -> Tuple[List[str], List[str]]:

        VALID_ANSWER_STYLES = ["all", "sentences", "multiple_choice"]

        if answer_style not in VALID_ANSWER_STYLES:
            raise ValueError(
                "Invalid answer style {}. Please choose from {}".format(
                    answer_style, VALID_ANSWER_STYLES
                )
            )
        global inputs
        inputs = []
        answers = []

        if answer_style == "sentences" or answer_style == "all":
            segments = self._split_into_segments(text)

            for segment in segments:
                sentences = self._split_text(segment)
                prepped_inputs, prepped_answers = self._prepare_qg_inputs(
                    sentences, segment
                )
                inputs.extend(prepped_inputs)
                answers.extend(prepped_answers)

        if answer_style == "multiple_choice" or answer_style == "all":
            sentences = self._split_text(text)
            prepped_inputs, prepped_answers = self._prepare_qg_inputs_MC(
                sentences
            )
            inputs.extend(prepped_inputs)
            answers.extend(prepped_answers)

        return inputs, answers

    def generate_questions_from_inputs(self, qg_inputs: List) -> List[str]:
        generated_questions = []

        for qg_input in qg_inputs:
            question = self._generate_question(qg_input)
            generated_questions.append(question)

        return generated_questions

    def _split_text(self, text: str) -> List[str]:
        MAX_SENTENCE_LEN = 128
        sentences = re.findall(".*?[.!\?]", text)
        cut_sentences = []

        for sentence in sentences:
            if len(sentence) > MAX_SENTENCE_LEN:
                cut_sentences.extend(re.split("[,;:)]", sentence))

        cut_sentences = [s for s in sentences if len(s.split(" ")) > 5]
        sentences = sentences + cut_sentences

        return list(set([s.strip(" ") for s in sentences]))

    def _split_into_segments(self, text: str) -> List[str]:

        MAX_TOKENS = 490
        paragraphs = text.split("\n")
        tokenized_paragraphs = [
            self.qg_tokenizer(p)["input_ids"] for p in paragraphs if len(p) > 0
        ]
        segments = []

        while len(tokenized_paragraphs) > 0:
            segment = []

            while len(segment) < MAX_TOKENS and len(tokenized_paragraphs) > 0:
                paragraph = tokenized_paragraphs.pop(0)
                segment.extend(paragraph)
            segments.append(segment)

        return [self.qg_tokenizer.decode(s, skip_special_tokens=True) for s in segments]

    def _prepare_qg_inputs(
        self,
        sentences: List[str],
        text: str
    ) -> Tuple[List[str], List[str]]:
       
        global inputs
        inputs = []
        answers = []

        for sentence in sentences:
            qg_input = f"{self.ANSWER_TOKEN} {sentence} {self.CONTEXT_TOKEN} {text}"
            inputs.append(qg_input)
            answers.append(sentence)

        return inputs, answers

    def _prepare_qg_inputs_MC(self, sentences: List[str]) -> Tuple[List[str], List[str]]:
        spacy_nlp = en_core_web_sm.load()
        docs = list(spacy_nlp.pipe(sentences, disable=["parser"]))
        global context
        context=[]
        inputs_from_text = []
        answers_from_text = []

        for doc, sentence in zip(docs, sentences):
            entities = doc.ents
            if entities:

                for entity in entities:
                    qg_input = f"{self.ANSWER_TOKEN} {entity} {self.CONTEXT_TOKEN} {sentence}"
                    qg_context= f"{sentence}"
                    answers = self._get_MC_answers(entity, docs)
                    inputs_from_text.append(qg_input)
                    context.append(qg_context)
                    answers_from_text.append(answers)

        return inputs_from_text, answers_from_text

    def _get_MC_answers(self, correct_answer: Any, docs: Any) -> List[Mapping[str, Any]]:

        entities = []

        for doc in docs:
            entities.extend([{"text": e.text, "label_": e.label_}
                            for e in doc.ents])

        # remove duplicate elements
        entities_json = [json.dumps(kv) for kv in entities]
        pool = set(entities_json)
        num_choices = (
            min(4, len(pool)) - 1
        )  # -1 because we already have the correct answer

        # add the correct answer
        final_choices = []
        correct_label = correct_answer.label_
        final_choices.append({"answer": correct_answer.text, "correct": True})
        pool.remove(
            json.dumps({"text": correct_answer.text,
                       "label_": correct_answer.label_})
        )

        # find answers with the same NER label
        matches = [e for e in pool if correct_label in e]

        # if we don't have enough then add some other random answers
        if len(matches) < num_choices:
            choices = matches
            pool = pool.difference(set(choices))
            choices.extend(random.sample(pool, num_choices - len(choices)))
        else:
            choices = random.sample(matches, num_choices)

        choices = [json.loads(s) for s in choices]

        for choice in choices:
            final_choices.append({"answer": choice["text"], "correct": False})

        random.shuffle(final_choices)
        return final_choices

    @torch.no_grad()
    def _generate_question(self, qg_input: str) -> str:

        encoded_input = self._encode_qg_input(qg_input)
        output = self.qg_model.generate(input_ids=encoded_input["input_ids"])
        question = self.qg_tokenizer.decode(
            output[0],
            skip_special_tokens=True
        )
        return question

    def _encode_qg_input(self, qg_input: str) -> torch.tensor:
      
        return self.qg_tokenizer(
            qg_input,
            padding='max_length',
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

    def _get_ranked_qa_pairs(
        self, generated_questions: List[str], qg_answers: List[str], scores, num_questions: int = 10
    ) -> List[Mapping[str, str]]:
        if num_questions > len(scores):
            num_questions = len(scores)
            print((
                f"\nWas only able to generate {num_questions} questions.",
                "For more questions, please input a longer text.")
            )

        qa_list = []

        for i in range(num_questions):
            index = scores[i]
            qa = {
                "question": generated_questions[index].split("?")[0] + "?",
                "answer": qg_answers[index],
                "context":context[index],
                "rank": None,
                "correct_answer" : None
            }
            qa_list.append(qa)

        return qa_list

    def _get_all_qa_pairs(self, generated_questions: List[str], qg_answers: List[str]):

        qa_list = []

        for question, answer in zip(generated_questions, qg_answers):
            qa = {
                "question": question.split("?")[0] + "?",
                "answer": answer
            }
            qa_list.append(qa)

        return qa_list


class QAEvaluator:

    def __init__(self) -> None:

        QAE_PRETRAINED = "iarfmoose/bert-base-cased-qa-evaluator"
        self.SEQ_LENGTH = 512

        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        self.qae_tokenizer = AutoTokenizer.from_pretrained(QAE_PRETRAINED)
        self.qae_model = AutoModelForSequenceClassification.from_pretrained(
            QAE_PRETRAINED
        )
        self.qae_model.to(self.device)
        self.qae_model.eval()

    def encode_qa_pairs(self, questions: List[str], answers: List[str]) -> List[torch.tensor]:
        encoded_pairs = []

        for question, answer in zip(questions, answers):
            encoded_qa = self._encode_qa(question, answer)
            encoded_pairs.append(encoded_qa.to(self.device))

        return encoded_pairs

    def get_scores(self, encoded_qa_pairs: List[torch.tensor]) -> List[float]:
        scores = {}

        for i in range(len(encoded_qa_pairs)):
            scores[i] = self._evaluate_qa(encoded_qa_pairs[i])

        return [
            k for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)
        ]

    def _encode_qa(self, question: str, answer: str) -> torch.tensor:
        if type(answer) is list:
            for a in answer:
                if a["correct"]:
                    correct_answer = a["answer"]
        else:
            correct_answer = answer

        return self.qae_tokenizer(
            text=question,
            text_pair=correct_answer,
            padding="max_length",
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        )

    @torch.no_grad()
    def _evaluate_qa(self, encoded_qa_pair: torch.tensor) -> float:
        output = self.qae_model(**encoded_qa_pair)
        return output[0][0][1]


def print_qa(qa_list: List[Mapping[str, str]], show_answers: bool = True) -> None:

    for i in range(len(qa_list)):
        # wider space for 2 digit q nums
        space = " " * int(np.where(i < 9, 3, 4))

        print(f"{i + 1}) Q: {qa_list[i]['question']}")

        answer = qa_list[i]["answer"]

        # print a list of multiple choice answers
        if type(answer) is list:

            if show_answers:
                print(
                    f"{space}A: 1. {answer[0]['answer']} "
                    f"{np.where(answer[0]['correct'], '(correct)', '')}"
                )
                for j in range(1, len(answer)): 
                    print(
                        f"{space + '   '}{j + 1}. {answer[j]['answer']} "
                        f"{np.where(answer[j]['correct']==True,'(correct)', '')}"
                    )

            else:
                print(f"{space}A: 1. {answer[0]['answer']}")
                for j in range(1, len(answer)):
                    print(f"{space + '   '}{j + 1}. {answer[j]['answer']}")

            print("")
       
        # print full sentence answers
        else:
            if show_answers:
                print(f"{space}A: {answer}\n")
        

In [4]:
if torch.cuda.is_available:
  print('GPU available')
else:
  print('Please set GPU via Edit -> Notebook Settings.')

GPU available


In [5]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"

In [6]:
model_name = "allenai/unifiedqa-t5-small" # you can specify the model size here
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [7]:
qg = QuestionGenerator()

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/121 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

## Import/Upload File

In [33]:
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/632c4e1934cc01a8a80dad82")
resp= request.json()
file_path=(resp["data"]["file_path"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])
user_choice_noa = (resp["data"]["is_none_of_above"]) 
needed_user_noa_questions = (resp["data"]["question_percentage"]) 

In [10]:
if file_type=="txt":
  import urllib.request
  response = urllib.request.urlopen(file_path)
  html = response.read()
  text=html.decode('utf8')
elif file_type=="doc":
  import urllib.request
  response = urllib.request.urlopen(file_path)
  html = response.read()
  text=html.decode('utf8')
elif file_type=="pdf":
  url = file_path
  response = requests.get(url)
  my_raw_data = response.content

  with open("my_pdf.pdf", 'wb') as my_data:
      my_data.write(my_raw_data)

  open_pdf_file = open("my_pdf.pdf", 'rb')
  read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
  NumPages = read_pdf.getNumPages()
  for i in range(0,NumPages):
    if read_pdf.isEncrypted:
        read_pdf.decrypt("")
        text = read_pdf.getPage(i).extractText()
    else:
        text = read_pdf.getPage(i).extractText()

elif file_type=="docx":
  url = file_path
  response = requests.get(url)
  my_raw_data = response.content
  with open("my_doc.txt", "wb") as text_file:
    text_file.write(my_raw_data)

  open_docx_file = open("my_doc.txt", 'rb')

  text = docx2txt.process(open_docx_file)
else:
  print("Invalid File Type")

## **Data Cleaning And Data Preprossing**

**Split text file into sentences**

In [11]:
sentences = sent_tokenize(text)
text_to_sentence= np.array(sentences)

**Split sentences into words by white space**

In [12]:
sentence_to_words=[]
for i in text_to_sentence[0:]:
    sentence_to_words.extend(i.split()) 

**Remove Unwanted Words** 

In [13]:
tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]

**Remove punctuation from each word(except full stop(.))**

In [14]:
remove = string.punctuation
remove = remove.replace(".", "")
pattern = r"[{}]".format(re.escape(remove))
table = str.maketrans('', '', pattern)
stripped = [w.translate(table) for w in tokens]


**Join Sentence**

In [15]:
words_to_sentense=' '.join(stripped)

In [16]:
MCQ =[]
qa_list = qg.generate(
    words_to_sentense, 
    num_questions= int(int(number_of_question)*3),
    answer_style= 'multiple_choice'
)
MCQ.append(qa_list)
print_qa(qa_list)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Evaluating QA pairs...

1) Q: What language did they sing in?
   A: 1. Prussians 
      2. German (correct)
      3. Prussian 
      4. French 

2) Q: What was the most beautiful language in the world?
   A: 1. Prussians 
      2. Sunday 
      3. one 
      4. French (correct)

3) Q: When was the last time I saw the commotion?
   A: 1. tomorrow 
      2. Sunday (correct)
      3. next day 
      4. the last two years 

4) Q: What language is the order to teach in Alsace?
   A: 1. German (correct)
      2. Prussians 
      3. French 
      4. Prussian 

5) Q: How long have we been in the army?
   A: 1. next day 
      2. tomorrow 
      3. the last two years (correct)
      4. forty years 

6) Q: What time did I get to my desk?
   A: 1. That day 
      2. French 
      3. Vive La France 
      4. morning (correct)

7) Q: what day is the essais?
   A: 1. the last two years 
      2. next day 
      3. tomorrow (correct)
      4. Sunday 

8) Q: how long had he been there?
   A: 1. next d

In [17]:
output = []
for j in MCQ:
  for k in j:
    row_data = {"question":k["question"],"options": [],"answer":"","context":k["context"], "rank": k["rank"], "correct_answer":k["correct_answer"]}
    for opt in k["answer"]:
      row_data["options"].append(opt["answer"])
      if opt["correct"] == True:
        row_data["answer"]=opt["answer"]
    output.append(row_data)

## Duplication check

In [18]:
new_questions = []
for i in output:
    if i not in new_questions:
        new_questions.append(i)

## Get Questions

In [19]:
scoring = []
for i in new_questions:
    questions_list = i.get("question")
    scoring.append(questions_list)   

## Get Rank of Questions

In [20]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = words_to_sentense
question_score = []
for i in scoring:
      final_score_list = question_answerer(question=i, context=context)
      final_score = final_score_list.get("score")
      final_score ="%.2f" % round(final_score*100, 2)
      question_score.append(final_score)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [21]:

for option_index, option in enumerate(new_questions):
    option["rank"] = question_score[option_index]

In [22]:
final_generated_questions = sorted(new_questions, key=lambda i: i['rank'],reverse=True)

## None of above replace with options

In [23]:
def add_noa_option(data,user_required_data):
    length_total_data = len(data)
    if user_required_data != "default":
        length_user_required_total_data = round((user_required_data*length_total_data)/100)
        user_data = random.sample(data, k=length_user_required_total_data)
        length_first_wrong_data = round((50*len(user_data))/100)
        first_wrong_data = random.sample(user_data, k=length_first_wrong_data)

        replaced_first_wrong_data = replace_wrong_data(first_wrong_data)

        second_right_data = []
        for i in user_data:
            if i not in first_wrong_data:
                second_right_data.append(i)
        replaced_second_right_data = replace_right_data(second_right_data)

        other_user_data = []
        for i in data:
            if i not in user_data:
                other_user_data.append(i)

        final_data = get_final_data(replaced_second_right_data, replaced_first_wrong_data, other_user_data)
        return final_data
    else:
        if length_total_data == 2:
            length_small_data  = (50*length_total_data)//100
            small_data_replace = random.sample(data, k=length_small_data)       
            get_replaced_small_wrong_data = replace_wrong_data(small_data_replace)

            small_other_data = []
            for i in data:
                if i not in small_data_replace:
                    small_other_data.append(i)

            final_data = get_final_data(list(), get_replaced_small_wrong_data, small_other_data)
            return final_data

        else:
            length_first_half_data = (40*length_total_data)//100
            data_replace = random.sample(data, k=length_first_half_data)

            if length_first_half_data == 1: 
                wrong_data = random.sample(data_replace, k=length_first_half_data)

                other_second_half = []
                for i in data:
                    if i not in data_replace:
                        other_second_half.append(i)

                get_replaced_second_wrong_data = replace_wrong_data(wrong_data)

                final_data = get_final_data(list(), get_replaced_second_wrong_data, other_second_half)
                return final_data

            else: 
                replace_half  = (50*length_first_half_data)//100 
                wrong_data = random.sample(data_replace, k=replace_half) 
                get_replaced_wrong_data = replace_wrong_data(wrong_data)

                right_data = []
                for i in data_replace:
                    if i not in wrong_data:
                        right_data.append(i)

                get_replaced_right_data = replace_right_data(right_data)

                other_data = []
                for i in data:
                    if i not in data_replace:
                        other_data.append(i)

                final_data = get_final_data(get_replaced_right_data, get_replaced_wrong_data, other_data)
                return final_data
        
def replace_right_data(data):
    for j in data:
        if "None of the above" not in j["options"]:
            for option_index, option in enumerate(j["options"]):
                if option.capitalize() == j["answer"] or option == j["answer"]:
                    j["options"][option_index] = "None of the above"
                    j["correct_answer"] = j["answer"]
                    j["answer"] =  j["options"][option_index]
    return data

def replace_wrong_data(data):
    for j in data:
        if "None of the above" not in j["options"]:
            for option_index, option in enumerate(j["options"]):
                option = random.choice(j["options"])
                if option.capitalize() != j["answer"] or option != j["answer"]:
                    j["options"][option_index] = "None of the above"
                    random.shuffle(j["options"])
                    break
        else:
            print("yes data present")
    return data

def get_final_data(data_right, data_wrong, data_other):
    final_data = data_right + data_wrong + data_other
    final_data = sorted(final_data, key=lambda i: i['rank'],reverse=True)
    return final_data

In [24]:
number_of_question = int(number_of_question)
if number_of_question > len(final_generated_questions) or number_of_question == 0: #
  print("please give value between 1 to {}".format(len(final_generated_questions)))
else:
    middle_index = number_of_question
    user_required_questions = final_generated_questions[:middle_index]
    other_questions = final_generated_questions[middle_index:]
   
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

    if user_choice_noa == "yes":
        user_required_questions = add_noa_option(user_required_questions,needed_user_noa_questions)
        other_questions = add_noa_option(other_questions,needed_user_noa_questions)

In [25]:
end = time.time()

In [26]:
from datetime import datetime

dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1

## **Final Output , User Validation & Post Generated Questions**

In [27]:
url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
headers = {'Content-Type':'application/json','Accept':'application/json'}
post_array ={
          "id" : _id,
          "questions" : user_required_questions,
          "other_questions" : other_questions,
          "upload_process_time": str(total_time)
      }
status = requests.post(url,headers=headers,data=json.dumps(post_array))
